__(a)__ Train a convolutional neural network on the data. Find a good set of hyperparameters for the model. Do you think a convolutional neural network is suitable for this kind of problem? Why/why not?

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


import numpy as np
from numpy.random import seed, randint

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [2]:
import os
os.path.abspath(os.getcwd())


'/'

In [3]:
X = np.load('/project/XSound.npy')
Y = tf.keras.utils.to_categorical(np.load('/project/YSound.npy'))
#Y = np.load('/project/YSound.npy')
print(X.shape)
print(Y.shape)
print(Y)

(15737, 62, 65, 1)
(15737, 4)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


In [4]:
X_, X_test, Y_, Y_test = train_test_split(X, Y, train_size=.8, random_state=504)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_, Y_, train_size=.75, random_state=504)
#X_train, y_train = train_test_split(X_train, Y_train)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(X_valid.shape)
print(Y_valid.shape)

(9441, 62, 65, 1)
(9441, 4)
(3148, 62, 65, 1)
(3148, 4)
(3148, 62, 65, 1)
(3148, 4)


In [5]:
print(X_train[0].shape)

(62, 65, 1)


In [36]:
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

cnn = tf.keras.models.Sequential([
    layers.Conv2D(240, 3, activation='relu', padding='same', input_shape=X_train[0].shape),
    layers.MaxPooling2D((3)),
    layers.SpatialDropout2D(.4),
    layers.AveragePooling2D(),
    layers.Conv2D(120, 3, activation='relu', padding='same'),
    layers.SpatialDropout2D(.4),
    layers.AveragePooling2D(),
    #layers.Conv2D(16, 3, activation='relu', padding='same'),
    #layers.SpatialDropout2D(.4),
    layers.Flatten(),
    layers.Dense(units=150, activation='relu'),
    layers.Dropout(.4),
    layers.Dense(units=4, activation='softmax')
])

cnn.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.SGD(learning_rate = 0.0001), metrics = ['accuracy'])
cnn.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 62, 65, 240)       2400      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 20, 21, 240)       0         
_________________________________________________________________
spatial_dropout2d_21 (Spatia (None, 20, 21, 240)       0         
_________________________________________________________________
average_pooling2d_18 (Averag (None, 10, 10, 240)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 10, 10, 120)       259320    
_________________________________________________________________
spatial_dropout2d_22 (Spatia (None, 10, 10, 120)       0         
_________________________________________________________________
average_pooling2d_19 (Averag (None, 5, 5, 120)       

In [37]:
history = cnn.fit(X_train, Y_train, 
                    epochs = 100, 
                    validation_data=(X_valid, Y_valid),
                    callbacks=[es])

Train on 9441 samples, validate on 3148 samples
Epoch 1/100
9441/9441 [==============================] - 3s 312us/sample - loss: 1.5047 - accuracy: 0.2457 - val_loss: 1.4386 - val_accuracy: 0.2446
Epoch 2/100
9441/9441 [==============================] - 3s 296us/sample - loss: 1.3870 - accuracy: 0.2542 - val_loss: 1.4392 - val_accuracy: 0.2481
Epoch 3/100
9441/9441 [==============================] - 3s 292us/sample - loss: 1.3863 - accuracy: 0.2632 - val_loss: 1.4388 - val_accuracy: 0.2402
Epoch 4/100
9441/9441 [==============================] - 3s 288us/sample - loss: 1.3871 - accuracy: 0.2479 - val_loss: 1.4386 - val_accuracy: 0.2405
Epoch 5/100
9441/9441 [==============================] - 3s 287us/sample - loss: 1.3865 - accuracy: 0.2490 - val_loss: 1.4397 - val_accuracy: 0.2408
Epoch 6/100
9441/9441 [==============================] - 3s 292us/sample - loss: 1.3862 - accuracy: 0.2592 - val_loss: 1.4392 - val_accuracy: 0.2370
Epoch 7/100
9441/9441 [==============================] - 3

In [35]:
train_eval = cnn.evaluate(X_train[:9441], Y_train[:9441])
test_eval = cnn.evaluate(X_test, Y_test)
print("On train data: Loss = {loss:.4f}, accuracy = {accuracy:.4f}".format(loss=train_eval[0], accuracy=train_eval[1]))
print("On test data: Loss = {loss:.4f}, accuracy = {accuracy:.4f}".format(loss=test_eval[0], accuracy=test_eval[1]))
print("Error on test data: {p:.2f}%".format(p = 100*(1-test_eval[1])))

3148/3148 [==============================] - 0s 72us/sample - loss: 1.8256 - accuracy: 0.2456
On train data: Loss = 1.8409, accuracy = 0.2449
On test data: Loss = 1.8256, accuracy = 0.2456
Error on test data: 75.44%


__(b)__ Classify instances of the test set using your models. Draw a confusion matrix and comment on the results.

__(c)__ Choose one other algorithm from the course, and redo (a) and (b) using this algorithm. Supply a brief discussion of why we would expect this algorithm to do better/worse than the CNN.

In [1]:
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline 
import numpy as np
from sklearn.model_selection import train_test_split

alphas = 10**np.linspace(10,-2,100)*0.5
#lasso = Lasso(max_iter = 10000) # Standard scale pipeline will fix!
#coefs = []

X = np.load('/project/XSound.npy')
#Y = tf.keras.utils.to_categorical(np.load('/project/YSound.npy'))
Y = np.load('/project/YSound.npy')
#print(X.shape)
#print(Y.shape)

X_df = pd.DataFrame(X.reshape(-1, 4030))
Y_df = pd.DataFrame(Y)
#print(Y_df.head)
#print(X_df.head)


X_train, X_test, Y_train, Y_test = train_test_split(X_df, Y_df, train_size=.8, random_state=0)
#X_train_df = df = pd.DataFrame(X_train.reshape(-1, 4030))
#Y_train_df = df = pd.DataFrame(Y_train)#.reshape(-1, len(Y_train)))
#X_test_df = df = pd.DataFrame(X_test.reshape(-1, len(X_test)))
#Y_test_df = df = pd.DataFrame(Y_test)#.reshape(-1, len(Y_test)))

#print(X[0:1])
#print(X_train.head)
#print(Y[0:1])
#print(Y_train.head)
# Create a pipeline with StandardScaler and RidgeCV
ridgecv = make_pipeline(StandardScaler(), RidgeCV(alphas=alphas, scoring='neg_mean_squared_error'))
ridgecv.fit(X_train, Y_train)

# Access the chosen alpha value from RidgeCV
chosen_alpha = ridgecv.named_steps['ridgecv'].alpha_
#print(chosen_alpha)
# Some of the coefficients
# 
# # Create a Ridge model using the chosen alpha from RidgeCV
ridge_model = Ridge(alpha=ridgecv.named_steps['ridgecv'].alpha_)
ridge_model.fit(X_train, Y_train)

# Calculate the test MSE using the trained Ridge model
mse = mean_squared_error(Y_test, ridge_model.predict(X_test))
print(mse) 
ridge_model.fit(X_df, Y_df)

0.8835193682117054


Ridge(alpha=53613.36110051616)